# Get adjectives related to male and female

In [1]:
import numpy as np
from itertools import combinations, filterfalse
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.keyedvectors import KeyedVectors
import pandas as pd
import random
import sys
import os
import pickle

import pandas as pd
from pandas.core import indexing
from gender_predictor.GenderClassifier import classify_gender
from collections import defaultdict
from tqdm.notebook import tqdm
import multiprocessing as mp
import json

In [2]:
movie_data = pd.read_csv('movie.metadata.tsv', sep='\t', skip_blank_lines=True, header=None, names=['id', 'free_id', 'movie_name', 'release_date', 'revenue', 'runtime', 'languages', 'countries', 'genres'])

In [3]:
character = pd.read_csv('character.metadata.tsv', sep='\t', skip_blank_lines=True, header=None, names=['id', 'free_id', 'release_date','char_name', 'dob', 'gender', 'height', 'ethnicity', 'name', 'age', 'free_char_id1', 'free_char_id2', 'free_char_id3'])
character = character[['id', 'char_name', 'gender']]

In [4]:
movie_data['release_year'] = movie_data['release_date'].apply(lambda r:r[:4] if str(r)!='nan' else None)

In [5]:
movie_id_by_year = {'United States of America':{}, 'India':{}}

for index, row in movie_data.iterrows():
    for key, value in json.loads(row['countries']).items():            
        if value == 'United States of America' or value == 'India':
            if row['release_year'] not in movie_id_by_year[value]:
                movie_id_by_year[value][row['release_year']] = [row.id]
            else:
                movie_id_by_year[value][row['release_year']].append(row.id)

In [23]:
count = [i for i in movie_id_by_year['India'].keys() if i is not None and i <'2000' and i>='1990']

In [24]:
decade_1990 = []
for c in count:
    decade_1990 += movie_id_by_year['India'][c]

In [25]:
len(decade_1990)

1235

In [26]:
female_words = ['she', 'her', 'woman', 'women', 'ladies', 'girls', 'lady', 'female', 'girl', 'damsel', 'maiden', 'daughter', 'sister', 'mother']
male_words   = ['he', 'his', 'man', 'male', 'men', 'boys', 'gentleman', 'gentlemen', 'boy', 'bloke', 'brother', 'father']


def read_input_file(filename):
    data_df = pd.read_csv(filename,sep=',', skip_blank_lines=True, index_col= False)
    return data_df


def get_plots_by_movie_id(data_df):
    movie_ids = data_df.movie_id.unique() 
    grouped = data_df.groupby(data_df.movie_id)

    all_movie_plots = []
    for id in movie_ids:
        sents_df = grouped.get_group(id)
        all_movie_plots.append(sents_df)
    return all_movie_plots


def get_frequency_for_movie(movie):
    
    if movie.iloc[0]['movie_id'] in decade_1990:
        return
    
    frequency_list = {'M':defaultdict(int), 'F':defaultdict(int)}
    name_data  = movie[((movie.dep_pos == 'NNP') & (movie.dep_ner == 'PERSON')) | (movie.dependent.isin(female_words)) | (movie.dependent.isin(male_words))]
    char_list  = character[character.id==movie.iloc[0]['movie_id']]
    
    gender_list = {}
    for idx,name in name_data.iterrows():            
        try:
            character_name = name['dependent'].lower()
            gender = None
            if character_name in gender_list:
                gender = gender_list[character_name]
            elif character_name in female_words:
                gender = 'F'
            elif character_name in male_words:
                gender = 'M'
            else:
                for ix, char in char_list.iterrows():
                    chk = str(char['char_name'])
                    if character_name in chk.lower():
                        gender = char['gender']
                        break
                    
                if gender is None:
                    gender = classify_gender(character_name)
                    
            gender_list[character_name] = gender
            governor = int(name['governor'])
            governor_df = movie[(movie['sentence_id']==name['sentence_id']) & (movie['token_id'] == governor) & (movie['dep_pos'].isin(['JJ', 'VB', 'VBP', 'VBZ', 'VBN']))]
            df2 = movie[(movie['sentence_id']==name['sentence_id']) & (movie['governor'] == name['token_id'])  & (movie['dep_pos'].isin(['JJ', 'VB', 'VBP', 'VBZ', 'VBN']))]
            df3 = movie[(movie['sentence_id'] == name['sentence_id']) & (movie['governor']==name['governor']) & (movie['dep_pos'].isin(['JJ', 'VB', 'VBP', 'VBZ', 'VBN']))]
            y = pd.concat([governor_df, df2, df3]).drop_duplicates()
            for i, x in y.iterrows():
                frequency_list[gender][x['dependent']] +=1
        except Exception as exc:
            pass
    print(movie.iloc[0]['movie_id'])
    print(frequency_list)
    return frequency_list
    
def get_name_and_adjective_mapping(all_movie_plots):
    frequency_list = {'M':defaultdict(int), 'F':defaultdict(int)}
    pool = mp.Pool(20)
    results = [pool.apply_async(get_frequency_for_movie, args=(movie,)) for movie in all_movie_plots]    
    output = [p.get() for p in results]
    return output


def get_adjective_cloud(filename):
    movie_data_df = read_input_file(filename)
    all_movie_plots = get_plots_by_movie_id(movie_data_df)
    name_adj_cluster_list = get_name_and_adjective_mapping(all_movie_plots)
    return name_adj_cluster_list



In [ ]:
%time result = get_adjective_cloud('india_lemma.csv')

20553214
{'M': defaultdict(<class 'int'>, {'steal': 2}), 'F': defaultdict(<class 'int'>, {'offer': 1, 'aid': 1, 'make': 1})}
Accuracy: 0.969757
Accuracy: 0.970225
26623942
{'M': defaultdict(<class 'int'>, {}), 'F': defaultdict(<class 'int'>, {'send': 1, 'be': 1, 'old': 1, 'pray': 1, 'appear': 1})}
Accuracy: 0.969653Accuracy: 0.968247

Accuracy: 0.967779Accuracy: 0.968091

Accuracy: 0.969549Accuracy: 0.969132
Accuracy: 0.968664

Accuracy: 0.970694Accuracy: 0.968195

Accuracy: 0.967779
Accuracy: 0.970225
32345990Accuracy: 0.967935
Accuracy: 0.969757Accuracy: 0.969236{'M': defaultdict(<class 'int'>, {'see': 2, 'decide': 1, 'do': 1, 'end': 1, 'approve': 1, 'know': 1}), 'F': defaultdict(<class 'int'>, {'turn': 1, 'rich': 1, 'spoil': 1, 'bratty': 1, 'young': 1, 'conceit': 1, 'attract': 1, 'complicate': 1, 'be': 1, 'see': 1, 'sing': 2, 'true': 1})}Accuracy: 0.968091
Accuracy: 0.968404




Accuracy: 0.970798
Accuracy: 0.966582
23454846
{'M': defaultdict(<class 'int'>, {'be': 10, 'have': 2, 'fo

Accuracy: 0.969913{'M': defaultdict(<class 'int'>, {'be': 3, 'dignified': 1, 'close': 2, 'relative': 3, 'send': 1, 'meet': 2, 'attract': 1, 'good': 1, 'take': 1, 'start': 1, 'find': 1, 'get': 1, 'new': 1, 'go': 1, 'play': 1, 'let': 1, 'know': 1}), 'F': defaultdict(<class 'int'>, {'be': 1, 'relative': 1, 'other': 1, 'new': 2, 'pious': 2, 'marry': 1})}
Accuracy: 0.971214

Accuracy: 0.968351
Accuracy: 0.969132
Accuracy: 0.972412
Accuracy: 0.969653
Accuracy: 0.967206
Accuracy: 0.968976
Accuracy: 0.968508
31690131
Accuracy: 0.969184{'M': defaultdict(<class 'int'>, {'common': 1}), 'F': defaultdict(<class 'int'>, {})}Accuracy: 0.971266


Accuracy: 0.969549
Accuracy: 0.971475
Accuracy: 0.968039
11353896
{'M': defaultdict(<class 'int'>, {'unmarried': 1, 'live': 1, 'inform': 3, 'clear': 2, 'have': 1, 'plan': 1, 'accommodate': 4, 'dependent': 1, 'be': 1}), 'F': defaultdict(<class 'int'>, {'live': 2, 'unmarried': 2, 'stay': 1, 'accommodate': 1, 'inform': 1, 'decide': 1, 'take': 1, 'extreme': 1})}


Accuracy: 0.970277
Accuracy: 0.969653
Accuracy: 0.970590
Accuracy: 0.967258
Accuracy: 0.972151
Accuracy: 0.970017
Accuracy: 0.968143
1094668
{'M': defaultdict(<class 'int'>, {'be': 9, 'figure': 1, 'withhold': 2, 'have': 1, 'defeat': 1, 'common': 1, 'meet': 2, 'fall': 1, 'little': 1, 'introduce': 1, 'true': 1, 'marry': 3, 'get': 1, 'receive': 2, 'unwell': 1, 'rush': 1, 'remove': 1, 'quell': 1, 'obey': 1, 'find': 2, 'inform': 1, 'unable': 1, 'slaughter': 1, 'heartbroken': 1, 'swear': 1, 'protect': 1, 'embark': 1, 'seek': 1, 'send': 1, 'assassinate': 2, 'save': 2, 'take': 1, 'care': 1, 'wild': 1, 'eliminate': 2, 'kill': 2, 'anger': 2, 'declare': 1, 'know': 2, 'do': 1, 'prepare': 1, 'visit': 1, 'discover': 1, 'suppose': 1, 'search': 1, 'apologize': 1, 'interrupt': 1, 'realize': 1, 'dead': 1}), 'F': defaultdict(<class 'int'>, {'be': 7, 'compel': 1, 'lovely': 2, 'meet': 2, 'fall': 1, 'little': 1, 'unwell': 1, 'fine': 1, 'find': 3, 'go': 1, 'have': 1, 'care': 1, 'kill': 1, 'know': 1, 'do': 1,

{'M': defaultdict(<class 'int'>, {'commit': 3, 'send': 1, 'reach': 1, 'get': 2, 'meet': 4, 'express': 1, 'fall': 1, 'show': 1, 'last': 1, 'sentence': 1, 'be': 3, 'happy': 1, 'hang': 1, 'act': 1, 'have': 1, 'reveal': 2, 'tense': 1, 'kill': 2, 'alive': 1, 'throw': 1, 'announce': 2, 'follow': 1, 'believe': 1, 'find': 2, 'extramarital': 1, 'ridiculous': 1, 'american': 1}), 'F': defaultdict(<class 'int'>, {'have': 1, 'start': 1, 'attend': 1, 'leave': 2, 'dead': 1, 'surprise': 1, 'be': 1, 'see': 1, 'go': 1, 'act': 1, 'kill': 4, 'run': 1, 'tell': 2, 'follow': 1, 'remember': 1, 'find': 1, 'unfold': 1})}
Accuracy: 0.968612
7567438
{'M': defaultdict(<class 'int'>, {'want': 1, 'distribute': 1, 'happen': 3, 'be': 1, 'identical': 1, 'have': 1, 'give': 1}), 'F': defaultdict(<class 'int'>, {'keep': 1, 'happen': 1})}
Accuracy: 0.969549
7566281
{'M': defaultdict(<class 'int'>, {'marry': 1, 'be': 2, 'have': 1, 'senior': 2, 'hand-in-glove': 1, 'realize': 1, 'rich': 1, 'deal': 1}), 'F': defaultdict(<class

Accuracy: 0.968247
Accuracy: 0.969861
Accuracy: 0.969705
Accuracy: 0.968299
Accuracy: 0.970382
Accuracy: 0.969393
Accuracy: 0.970069Accuracy: 0.970277Accuracy: 0.969497


28350035
{'M': defaultdict(<class 'int'>, {'like': 2, 'do': 1, 'promise': 1, 'commit': 1, 'run': 1, 'dead': 1, 'be': 1, 'return': 1, 'marry': 1}), 'F': defaultdict(<class 'int'>, {'be': 5, 'grand': 1, 'love': 1, 'other': 1, 'big': 1, 'marry': 1, 'like': 1})}
Accuracy: 0.969653
Accuracy: 0.969288
28275468
{'M': defaultdict(<class 'int'>, {'develop': 1, 'be': 1, 'lead': 1}), 'F': defaultdict(<class 'int'>, {})}
Accuracy: 0.969028
Accuracy: 0.969601
Accuracy: 0.968768
Accuracy: 0.969288
Accuracy: 0.969028
28066754
{'M': defaultdict(<class 'int'>, {'save': 4, 'be': 9, 'kill': 3, 'visit': 1, 'meet': 3, 'respond': 1, 'ask': 3, 'make': 1, 'hurt': 1, 'see': 1, 'keen': 1, 'happy': 1, 'have': 4, 'turn': 1, 'old': 2, 'try': 1, 'tell': 2, 'catch': 1, 'assault': 1, 'own': 1, 'begin': 1, 'revenge': 1, 'live': 1, 'beat': 1, 'find': 

...

{'M': defaultdict(<class 'int'>, {'get': 1, 'spiritual': 1, 'tell': 1, 'be': 1, 'real': 1, 'local': 1, 'next': 1, 'come': 1, 'raise': 1, 'hear': 1}), 'F': defaultdict(<class 'int'>, {'raise': 1, 'have': 1, 'be': 1, 'decide': 1, 'tell': 1, 'leave': 1, 'marry': 2, 'engage': 2, 'promiscuous': 1, 'indian': 1, 'name': 2, 'reconsider': 1})}
Accuracy: 0.969653
Accuracy: 0.971214
Accuracy: 0.969965
Accuracy: 0.968039
Accuracy: 0.968768
28193880
{'M': defaultdict(<class 'int'>, {'return': 1, 'retrieve': 1}), 'F': defaultdict(<class 'int'>, {'be': 1, 'have': 2, 'require': 1, 'bed-ridden': 1, 'work': 1, 'support': 1, 'old': 1, 'take': 1, 'save': 1, 'share': 1, 'understand': 1, 'let': 1})}
Accuracy: 0.968820
Accuracy: 0.970642Accuracy: 0.971735

7601618
{'M': defaultdict(<class 'int'>, {'be': 2, 'unquestionable': 1, 'have': 1, 'devote': 2, 'drive': 1, 'build': 1}), 'F': defaultdict(<class 'int'>, {'crazy': 1, 'sell': 2, 'be': 1, 'have': 1})}
Accuracy: 0.966373
11481973
{'M': defaultdict(<class 'in

{'M': defaultdict(<class 'int'>, {'alive': 1, 'be': 14, 'find': 4, 'name': 1, 'catch': 1, 'call': 2, 'meet': 1, 'get': 1, 'say': 2, 'make': 1, 'tell': 6, 'massacre': 2, 'acknowledge': 1, 'learn': 1, 'shocking': 1, 'think': 1, 'emotional': 1, 'dead': 4, 'try': 4, 'stop': 1, 'fall': 1, 'shoot': 5, 'own': 2, 'kill': 9, 'young': 1, 'hold': 1, 'return': 1, 'native': 1, 'welcome': 1, 'engage': 1, 'have': 1, 'change': 1, 'come': 1, 'transfer': 1, 'counter': 1, 'set': 1, 'chase': 1, 'attempt': 1, 'intervene': 1, 'see': 1, 'surrender': 1, 'willing': 1, 'give': 3, 'surprise': 1, 'know': 2, 'open': 2, 'spot': 1, 'hit': 1, 'go': 3, 'tragic': 1, 'assign': 2, 'save': 1, 'claim': 1, 'realize': 1, 'much': 1, 'witness': 1, 'do': 1, 'threaten': 1, 'take': 2, 'severe': 1, 'vow': 1, 'collapse': 1, 'rescue': 1, 'treat': 1}), 'F': defaultdict(<class 'int'>, {'be': 5, 'learn': 1, 'take': 2, 'swim': 1, 'try': 2, 'knock': 1, 'slit': 1, 'alive': 1, 'own': 1, 'discover': 1, 'say': 1, 'intervene': 2, 'tell': 1, '

{'M': defaultdict(<class 'int'>, {'send': 1, 'be': 2, 'deal': 1, 'official': 1, 'say': 2, 'get': 1, 'leave': 2, 'have': 1, 'reach': 1, 'shock': 1, 'protect': 1, 'learn': 1, 'only': 1, 'meet': 1, 'former': 1, 'kill': 1}), 'F': defaultdict(<class 'int'>, {'know': 1, 'be': 2, 'indian': 1, 'meet': 1, 'fail': 1, 'end': 1, 'realize': 1, 'leave': 1, 'protect': 1, 'think': 1, 'try': 1, 'eliminate': 1, 'save': 1, 'only': 1, 'settle': 1, 'accept': 2})}
33541530
{'M': defaultdict(<class 'int'>, {'lecherous': 2}), 'F': defaultdict(<class 'int'>, {})}
Accuracy: 0.969809
Accuracy: 0.970642
Accuracy: 0.969497
Accuracy: 0.970642
Accuracy: 0.968768
Accuracy: 0.968039Accuracy: 0.969393

Accuracy: 0.971162
Accuracy: 0.969861
Accuracy: 0.970590
3863665
{'M': defaultdict(<class 'int'>, {'give': 1, 'small': 1, 'name': 1, 'care': 1, 'learn': 2, 'attack': 2, 'beat': 3, 'tell': 5, 'hold': 3, 'love': 1, 'find': 3, 'have': 5, 'say': 3, 'like': 3, 'return': 1, 'do': 4, 'perform': 1, 'come': 1, 'meet': 4, 'marry':

{'M': defaultdict(<class 'int'>, {'marry': 1, 'good-for-nothing': 1, 'arrest': 1, 'realize': 2, 'be': 2}), 'F': defaultdict(<class 'int'>, {'be': 3, 'love': 1, 'unknown': 1, 'find': 3, 'decide': 1, 'sacrifice': 1, 'marry': 1, 'misunderstand': 1, 'begin': 1, 'suspect': 1, 'feel': 1, 'give': 1, 'pass': 1})}
Accuracy: 0.968091
Accuracy: 0.969393
Accuracy: 0.968039
Accuracy: 0.971319
34501845Accuracy: 0.968924
{'M': defaultdict(<class 'int'>, {'owe': 1, 'native': 1, 'doting': 1, 'old': 1, 'have': 1, 'become': 1, 'be': 1, 'marry': 1, 'happy': 1, 'free': 1, 'try': 1, 'recover': 1, 's': 1}), 'F': defaultdict(<class 'int'>, {'s': 3, 'end': 1, 'doting': 1, 'able': 1, 'be': 1, 'cover': 1, 'try': 1})}

Accuracy: 0.968456
36057872
{'M': defaultdict(<class 'int'>, {'male': 2, 'call': 1, 'begin': 1}), 'F': defaultdict(<class 'int'>, {'reveal': 1, 'marry': 1, 'get': 1})}
Accuracy: 0.969549
Accuracy: 0.967987
Accuracy: 0.967467
Accuracy: 0.970225
Accuracy: 0.970329
Accuracy: 0.969497
Accuracy: 0.96996

Accuracy: 0.970538
Accuracy: 0.970329
21855963
{'M': defaultdict(<class 'int'>, {'know': 2, 'be': 2, 'bring': 2, 'cure': 1, 'take': 2, 'human': 1, 'return': 2, 'beat': 1, 'go': 3, 'unable': 1, 'involve': 1, 'personal': 1, 'do': 1, 'rehearse': 1, 'portray': 1, 'find': 2, 'lose': 1, 'close': 1, 'happen': 1, 'real': 1, 'meet': 2, 'ask': 1, 'invite': 1, 'realize': 1}), 'F': defaultdict(<class 'int'>, {'tell': 1, 'do': 1, 'be': 1, 'lead': 1, 'female': 1, 'muslim': 1, 'leave': 1, 'take': 1, 'lose': 1, 'mental': 1, 'invite': 1, 'ask': 1, 'go': 1})}
Accuracy: 0.971631
Accuracy: 0.967467
Accuracy: 0.968768
Accuracy: 0.969653
Accuracy: 0.968351
11217695
{'M': defaultdict(<class 'int'>, {}), 'F': defaultdict(<class 'int'>, {'discover': 1})}
Accuracy: 0.968820
Accuracy: 0.969445
Accuracy: 0.967987
Accuracy: 0.968299
Accuracy: 0.971266
Accuracy: 0.969757
Accuracy: 0.970538
Accuracy: 0.968404
31156887
{'M': defaultdict(<class 'int'>, {}), 'F': defaultdict(<class 'int'>, {'be': 1, 'get': 1})}
Accurac

In [24]:
x1 = result

In [15]:
## Save frequency
import pickle
output = open('india_2000_freq_1.pkl', 'wb')
pickle.dump(frequency_list, output)
output.close()

In [18]:
import pickle
pkl_file = open('india_2000_freq_1.pkl', 'rb')

frequency_list = pickle.load(pkl_file)


In [21]:
# Combine frequency from different results
frequency_list = {'M':defaultdict(int),'F':defaultdict(int)}

for freq in result:
    if freq is None:
        continue
    for k, v in freq['M'].items():
        frequency_list['M'][k]+=v
    for k, v in freq['F'].items():
        frequency_list['F'][k]+=v  

# Calculate Odds Ratio

In [22]:
odds_ratio = {}
threshold  = 2
topk       = 50

total_num_f = sum(frequency_list['F'].values())
total_num_m = sum(frequency_list['M'].values())

for key in frequency_list['F'].keys():
    m_num = frequency_list['M'][key]
    f_num = frequency_list['F'][key]
    non_f_num = total_num_f - f_num
    non_m_num = total_num_m - m_num
    if f_num >= threshold and m_num >= threshold:
        # we only consider the events where there are at least {thresohld} occurences for both gender
        odds_ratio[key] = round((m_num / f_num) / (non_m_num / non_f_num), 2)
    else:
        continue

In [23]:
from operator import itemgetter

dict(sorted(odds_ratio.items(), key=itemgetter(1), reverse=True)[:topk]), dict(sorted(odds_ratio.items(), key=itemgetter(1))[:topk])

({'senior': 5.44,
  'old': 3.05,
  'terrorist': 2.77,
  'ready': 2.77,
  'responsible': 2.56,
  'drunk': 2.55,
  'sure': 2.55,
  'entire': 2.55,
  'good': 2.47,
  'due': 2.39,
  'wrong': 2.34,
  'successful': 2.34,
  'last': 2.3,
  'local': 2.23,
  'unknown': 2.23,
  'major': 2.02,
  'present': 1.7,
  'true': 1.6,
  'small': 1.59,
  'evil': 1.59,
  'happy-go-lucky': 1.59,
  'maternal': 1.59,
  'rival': 1.59,
  'high': 1.59,
  'final': 1.59,
  'suspicious': 1.49,
  'free': 1.49,
  'new': 1.45,
  'real': 1.41,
  'rich': 1.36,
  'muslim': 1.35,
  'much': 1.35,
  'unable': 1.31,
  'second': 1.27,
  'helpless': 1.27,
  'main': 1.27,
  'arrogant': 1.27,
  'anxious': 1.27,
  'hard': 1.27,
  'daily': 1.27,
  'ashamed': 1.27,
  'maya': 1.27,
  'alcoholic': 1.27,
  'past': 1.27,
  'notorious': 1.27,
  'clear': 1.27,
  'afraid': 1.27,
  'dev': 1.27,
  'mock': 1.27,
  'native': 1.27},
 {'beautiful': 0.09,
  'female': 0.11,
  'pregnant': 0.23,
  'biological': 0.27,
  'ill': 0.29,
  'teenage': 0.32,

In [67]:
top_m

['fumbles',
 'bowl',
 'common',
 'different',
 'avoid',
 'suffers',
 'acquitted',
 'melodious',
 'breaks',
 'born']

# Weat Score Calculation

In [ ]:
def swAB(W, A, B):
    """Calculates differential cosine-similarity between word vectors in W, A and W, B
     Arguments
              W, A, B : n x d matrix of word embeddings stored row wise
    """
    WA = cosine_similarity(W,A)
    WB = cosine_similarity(W,B)

    #Take mean along columns
    WAmean = np.mean(WA, axis = 1)
    WBmean = np.mean(WB, axis = 1)

    return (WAmean - WBmean)
  
def test_statistic(X, Y, A, B):
    """Calculates test-statistic between the pair of association words and target words
     Arguments
              X, Y, A, B : n x d matrix of word embeddings stored row wise
     Returns
              Test Statistic
    """
    return (sum(swAB(X, A, B)) - sum(swAB(Y, A, B)))

In [ ]:
def weat_effect_size(X, Y, A, B, embd):
    """Computes the effect size for the given list of association and target word pairs
     Arguments
              X, Y : List of association words
              A, B : List of target words
              embd : Dictonary of word-to-embedding for all words
     Returns
              Effect Size
    """

    Xmat = np.array([embd[w.lower()] for w in X if w.lower() in embd])
    Ymat = np.array([embd[w.lower()] for w in Y if w.lower() in embd])
    Amat = np.array([embd[w.lower()] for w in A if w.lower() in embd])
    Bmat = np.array([embd[w.lower()] for w in B if w.lower() in embd])

    XuY = list(set(X).union(Y))
    XuYmat = []
    for w in XuY:
    if w.lower() in embd:
        XuYmat.append(embd[w.lower()])
    XuYmat = np.array(XuYmat)


    d = (np.mean(swAB(Xmat,Amat,Bmat)) - np.mean(swAB(Ymat,Amat,Bmat)))/np.std(swAB(XuYmat, Amat, Bmat))

    return d

In [ ]:
def random_permutation(iterable, r=None):
    """Returns a random permutation for any iterable object"""
    pool = tuple(iterable)
    r = len(pool) if r is None else r
    return tuple(random.sample(pool, r))

def weat_p_value(X, Y, A, B, embd, sample = 1000):
    """Computes the one-sided P value for the given list of association and target word pairs
     Arguments
              X, Y : List of association words
              A, B : List of target words
              embd : Dictonary of word-to-embedding for all words
              sample : Number of random permutations used.
     Returns
    """
    size_of_permutation = min(len(X), len(Y))
    X_Y = X + Y
    test_stats_over_permutation = []

    Xmat = np.array([embd[w.lower()] for w in X if w.lower() in embd])
    Ymat = np.array([embd[w.lower()] for w in Y if w.lower() in embd])
    Amat = np.array([embd[w.lower()] for w in A if w.lower() in embd])
    Bmat = np.array([embd[w.lower()] for w in B if w.lower() in embd])

    if not sample:
        permutations = combinations(X_Y, size_of_permutation)
    else:
        permutations = [random_permutation(X_Y, size_of_permutation) for s in range(sample)]

    for Xi in permutations:
    Yi = filterfalse(lambda w:w in Xi, X_Y)
    Ximat = np.array([embd[w.lower()] for w in Xi if w.lower() in embd])
    Yimat = np.array([embd[w.lower()] for w in Yi if w.lower() in embd])
    test_stats_over_permutation.append(test_statistic(Ximat, Yimat, Amat, Bmat))

    unperturbed = test_statistic(Xmat, Ymat, Amat, Bmat)
  
  is_over = np.array([o > unperturbed for o in test_stats_over_permutation])
  
  return is_over.sum() / is_over.size

In [ ]:
"""List of association and target word pairs for the sample test (top_m, top_f) vs (Pleasant, Unpleasant)"""


X = top_m

Y = top_f

# Find better pleasant and unpleasant words

A = ["caress", "freedom", "health", "love", "peace", "cheer", "friend", "heaven", "loyal", "pleasure", "diamond", "gentle", "honest", "lucky", "rainbow", "diploma", "gift", "honor", "miracle", "sunrise", "family",
    "happy", "laughter", "paradise", "vacation"] 

B = ["abuse", "crash", "filth", "murder", "sickness", "accident", "death", "grief", "poison", "stink", "assault", "disaster", "hatred", "pollute", "tragedy", "divorce", "jail", "poverty", "ugly", "cancer", "kill", "rotten",
    "vomit", "agony", "prison"] 


resourceFile = ''
glove = KeyedVectors.load_word2vec_format(resourceFile + 'gensim_glove.840B.300d.txt.bin', binary=True)
print('The glove embedding has been loaded!')

"""Compute the effect-size and P value"""
print('WEAT d = ', weat_effect_size(X, Y, A, B, glove))
print('WEAT p = ', weat_p_value(X, Y, A, B, glove, 1000))